In [ ]:
import yfinance as yf
import pandas as pd
import sklearn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import scipy.stats as stats
import scipy
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Loading in the individual component data
components = ['NVDA', 'TSM', 'AVGO', 'AMD', 'TXN', 'QCOM',
    'AMAT', 'ASML', 'MU', 'ADI', 'LRCX', 'INTC', 'KLAC',
    'SNPS', 'CDNS', 'MRVL', 'NXPI', 'MCHP', 'MPWR', 'ON',
    'STM', 'TER', 'SWKS', 'OLED', 'QRVO']
data = {}
data = yf.download(components, start="2022-01-01", end="2023-01-01", group_by='ticker')
# Separate each type of data
adj_close_prices = data.xs('Adj Close', level=1, axis=1)
adj_close_prices = adj_close_prices.drop('LRCX', axis=1)

# Calculate daily returns based on adjusted close prices
returns = adj_close_prices.pct_change().dropna()

[***************       32%                       ]  8 of 25 completedERROR:yfinance:Could not get exchangeTimezoneName for ticker 'LRCX' reason: 'chart'
[*********************100%***********************]  25 of 25 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['LRCX']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


In [ ]:
#Adding manual weights for each ticker
weights = {}
weights['NVDA'] = 0.224
weights['TSM'] = 0.1346
weights['AVGO'] = 0.0768
weights['AMD'] = 0.0478
weights['TXN'] = 0.0429
weights['QCOM'] = 0.0424
weights['AMAT'] = 0.0415
weights['ASML'] = 0.0415
weights['MU'] = 0.0414
weights['ADI'] = 0.0399
weights['LRCX'] = 0.037
weights['INTC'] = 0.036
weights['KLAC'] = 0.0333
weights['SNPS'] = 0.0287
weights['CDNS'] = 0.025
weights['MRVL'] = 0.0226
weights['NXPI'] = 0.0191
weights['MCHP'] = 0.0146
weights['MPWR'] = 0.0145
weights['ON'] = 0.009
weights['STM'] = 0.0085
weights['TER'] = 0.0059
weights['SWKS'] = 0.0054
weights['OLED'] = 0.0036
weights['QRVO'] = 0.0033

weights = pd.Series(weights)
weights = weights[returns.columns]

In [ ]:
etf_price = (adj_close_prices * weights).sum(axis=1)
etf_returns = etf_price.pct_change().dropna()

In [ ]:
#Creating covariance matrix
cov_matrix = returns.cov()

In [ ]:
# Calculating volatility for the portfolio
portfolio_variance = weights.dot(cov_matrix).dot(weights)
portfolio_volatility = np.sqrt(portfolio_variance)

In [ ]:
print(f"Portfolio Variance: {portfolio_variance:.6f}")
print(f"Portfolio Volatility: {portfolio_volatility:.6f}")

Portfolio Variance: 0.000728
Portfolio Volatility: 0.026988


In [ ]:
portfolio_returns = (returns * weights).sum(axis=1)
#print(portfolio_returns)

In [ ]:
# Define the black scholes model to price options
def black_scholes_call(S, X, T, r, sigma):
    """
    Calculate the European Call option price using the Black-Scholes formula

    S: Current stock price (ETF price)
    X: Strike price
    T: Time to expiration (in years)
    r: Risk-free interest rate (annual)
    sigma: Volatility (annualized)
    """
    d1 = (np.log(S / X) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    print(d1, d2)

    call_price = S * norm.cdf(d1) - X * np.exp(-r * T) * norm.cdf(d2)
    return call_price

In [ ]:
S = etf_price[-1]  # Last price of the ETF (current price)
X = 80  # Example strike price (adjust this to your target strike price)
T = 30/252  # Time to expiration (e.g., 30 days, convert to years)
r = 0.02  # Risk-free rate (e.g., 2% annually)
sigma = portfolio_volatility  # Use the calculated portfolio volatility

<ipython-input-42-205f1dc0714a>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  S = etf_price[-1]  # Last price of the ETF (current price)


In [ ]:
# Running the black scholes model
call_price = black_scholes_call(S, X, T, r, sigma)
print(f"The European Call option price on the ETF is: ${call_price:.2f}")

29.233130208139034 29.22381851474094
The European Call option price on the ETF is: $24.96


In [ ]:
# PCA code
scaler = StandardScaler()
cov_matrix_scaled = scaler.fit_transform(cov_matrix)

pca = PCA(n_components=24)
pca.fit(cov_matrix_scaled)

principal_components = pca.components_
explained_variance = pca.explained_variance_ratio_
pca_df = pd.DataFrame(principal_components, columns=cov_matrix.columns)

In [ ]:
# Print the PCA data
pca_df

Ticker,CDNS,KLAC,MRVL,STM,ON,OLED,AMD,AMAT,MPWR,ADI,...,INTC,QCOM,SNPS,QRVO,NXPI,TXN,TER,TSM,AVGO,MU
0,0.178501,0.210213,0.211000,0.206981,0.208446,0.190152,0.206511,0.210121,0.207705,0.211685,...,0.196213,0.205830,0.181015,0.206289,0.209434,0.209009,0.204165,0.195784,0.209500,0.198982
1,0.620981,0.009317,0.033383,-0.041611,-0.090105,0.080434,0.077997,-0.086977,0.110903,-0.011398,...,-0.165002,-0.075857,0.599891,-0.131239,-0.103481,-0.131452,-0.130978,-0.115003,-0.038818,-0.237487
2,0.008515,-0.141462,0.033178,-0.274310,-0.035337,0.627181,-0.045787,-0.094416,-0.048604,-0.074401,...,0.314780,0.036249,-0.003250,0.227224,0.038730,-0.014887,0.027430,-0.460000,0.081446,0.012187
3,0.093299,-0.000643,-0.081545,-0.052068,-0.133497,-0.397250,0.281148,0.073220,-0.002757,-0.122488,...,0.627556,-0.071721,0.050589,-0.102030,-0.143771,-0.172713,0.210335,-0.186837,-0.060996,0.330355
4,-0.067267,0.122420,-0.044777,-0.115149,-0.182790,0.472121,0.027730,0.110157,0.118434,-0.180935,...,0.014620,-0.154464,-0.052643,-0.085146,-0.213087,-0.281948,0.340153,0.558303,-0.162012,0.001071
5,0.003535,-0.308492,0.171711,-0.220746,-0.008392,-0.122374,0.195398,-0.284736,0.164719,-0.045128,...,0.054738,0.369262,-0.020631,0.131897,-0.043417,-0.152467,-0.392963,0.440625,0.114743,0.125593
6,0.104467,-0.000213,-0.183857,0.120198,-0.041932,-0.025376,-0.398232,0.027472,0.099293,-0.069910,...,-0.268920,0.156756,0.193205,0.257953,-0.104260,-0.161038,0.025465,-0.069250,-0.194738,0.652381
7,0.086930,0.004034,-0.056149,-0.141743,-0.080622,-0.193486,-0.222626,0.052533,-0.313146,-0.061397,...,0.172084,0.529277,0.131497,0.245158,-0.039266,-0.057040,0.295826,0.111767,-0.073106,-0.434863
8,-0.065313,-0.236517,-0.088458,0.453406,-0.157541,0.261962,0.152645,-0.017768,-0.342066,-0.055309,...,0.132441,0.059138,-0.023749,0.068061,-0.086870,-0.035879,-0.406250,0.000652,-0.165970,0.009310
9,0.134698,-0.128069,-0.253123,0.100470,0.599483,0.131026,0.010336,-0.048353,-0.221580,-0.008035,...,0.205438,0.056996,0.107484,-0.220820,0.282108,-0.098337,-0.012820,0.182658,-0.285162,0.070734
